In [1]:
from gen_finetune.run_finetune_experiment import get_dataset, prep_train_dataset
from pathlib import Path
import json
import transformers
from transformers import pipeline
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version used by PyTorch: {torch.version.cuda}")

torch.cuda.init()
print("After init - CUDA available:", torch.cuda.is_available())

%reload_ext autoreload
%autoreload 2


PyTorch version: 2.7.1+cu126
CUDA available: True
CUDA version used by PyTorch: 12.6
After init - CUDA available: True


In [10]:
data_folder = Path("data/title_and_first_sen")
dataset, task_description = get_dataset(str(data_folder / "data-test.jsonl"), str(data_folder / "data-task.json"))
print(dataset)
print(dataset[0]["task_input_a"])
print(dataset[0]["task_answer_a"])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['task_input_a', 'task_input_b', 'task_answer_a', 'task_answer_b'],
    num_rows: 2601
})
System: You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI. ---- User: [Update] My family was robbed and we know who did it. How do I comfort my family and confront the robber? This is an update to my original post, found here:
https://www
Assistant: <reddit>relationships</reddit>


In [11]:
pipeline = pipeline(
    task="text-generation",
    model="huggyllama/llama-7b",
    torch_dtype=torch.float16,
    device=0
)
pipeline("Plants create energy through a process known as")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Device set to use cuda:0


[{'generated_text': 'Plants create energy through a process known as photosynthesis. The energy from the sun allows plants to make food. This food is made up of simple sugars that can be used as energy for the plant or to grow.\nTrees make their own food right inside their leaves through the process of photosynthesis. Their leaves have the job of collecting sunlight and turning it into food. To do this, trees have tiny structures in the upper surface of the leaves called stomata. Stomata are like tiny mouths. Plants need carbon dioxide to make food, so stomata open and close to allow carbon dioxide into the leaf and to release oxygen.\nIn the process of photosynthesis, the energy in the sunlight is converted into chemical energy. This energy is stored in the simple sugars that are produced. The sugars are used as food for the tree. They may also be used to make new leaves and branches.\nPlants can also create energy by eating other organisms. Ants, for example, can eat dead insects and

In [13]:
print(task_description)
print(dataset)
prepped_data = prep_train_dataset(dataset, task_description)

TaskDescription(prompt_a='Which subreddit does this belong to? -----', prompt_b='Continue the story. -----', tag_a='reddit', tag_b='story')
Dataset({
    features: ['task_input_a', 'task_input_b', 'task_answer_a', 'task_answer_b'],
    num_rows: 2601
})
